In [41]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
from selenium.common.exceptions import NoSuchElementException

In [45]:
# Prompt the user to input their LinkedIn username and password
USERNAME = input("Enter your LinkedIn username: ")
PASSWORD = input("Enter your LinkedIn password: ")

# Set up the Selenium WebDriver using ChromeDriver
driver = webdriver.Chrome()
driver.get("https://linkedin.com/uas/login")  # Open LinkedIn login page

# Find and fill in the username field
username_field = driver.find_element(By.ID, "username")
username_field.send_keys(USERNAME)
# Find and fill in the password field
password_field = driver.find_element(By.ID, "password")
password_field.send_keys(PASSWORD)

# Click on the sign-in button
sign_in_btn = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
sign_in_btn.click()
driver.maximize_window()  # Maximize the browser window for better visibility


In [2]:

# Set up the Selenium WebDriver using ChromeDriver
driver = webdriver.Chrome()
driver.get("https://linkedin.com/uas/login")  # Open LinkedIn login page

# Find and fill in the username field
username_field = driver.find_element(By.ID, "username")
username_field.send_keys("your_email")

# Find and fill in the password field
password_field = driver.find_element(By.ID, "password")
password_field.send_keys("your_password")

# Click on the sign-in button
sign_in_btn = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
sign_in_btn.click()

# Assuming the click was successful if no exception is raised
print("Sign-in button clicked successfully")

driver.maximize_window()  # Maximize the browser window for better visibility


Sign-in button clicked successfully


In [35]:
# Define the profile URL to scrape
profile_url = "https://www.linkedin.com/my-items/posted-jobs/?jobState=CLOSED"
driver.get(profile_url)  # Open the profile URL
print("Closed Job postings button clicked successfully")


Closed Job postings button clicked successfully


In [37]:
# Find the link for a specific job
link = driver.find_element(By.XPATH, '//a[@href="https://www.linkedin.com/hiring/jobs/3785030073/detail/"]')
link.click()  # Click on the job link
print("Virtual Assistant job position opened successfully")


Virtual Assistant job position opened successfully


In [38]:
# Click on the 'View applicants' button
span_element = driver.find_element(By.XPATH, '//span[text()="View applicants"]')
span_element.click()
print("View applicants button clicked successfully")


View applicants button clicked successfully


In [39]:

# Create a new BeautifulSoup object with updated page source
soup = BeautifulSoup(driver.page_source, 'lxml')
# Extract the number of pages from pagination
spans = soup.select('ul.artdeco-pagination__pages--number span')
total_pages = int(spans[-1].text)
print(f'Total pages for this job application are {total_pages}')

Total pages for this job application are 92


In [44]:
# Initialize variables
applicants_info = []

# Loop through each page of applicants
for i in range(total_pages):
    current_url = f"{driver.current_url}&start={i * 25}"  # Update URL with pagination
    print(current_url)
    driver.get(current_url)
    time.sleep(1)
    # Loop through each applicant on the page
    for j in range(1, 26):
        xpath = f"/html/body/div[4]/div[3]/div/div[3]/div/div/div[2]/ul/li[{j}]"
        try:
            link_element = driver.find_element(By.XPATH, xpath)
            link_element.click()
        except NoSuchElementException:
            print('No element found')
            continue
        if xpath:
            try:
                name = driver.find_element(By.XPATH, f"/html/body/div[4]/div[3]/div/div[3]/div/div/div[2]/ul/li[{j}]/a/div/div[2]/div[1]").text
            except NoSuchElementException:
                name = None
            time.sleep(1)
            # Extract applicant information
            try:
                current_title = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div/div[3]/div/main/div/div[2]/div[1]/section[1]/ul/li[1]/div/p[1]').text
            except NoSuchElementException:
                current_title = None
            try:
                current_company = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div/div[3]/div/main/div/div[2]/div[1]/section[1]/ul/li[1]/div/p[2]').text
            except NoSuchElementException:
                current_company = None
            try:
                location = driver.find_element(By.XPATH, f'/html/body/div[4]/div[3]/div/div[3]/div/main/div/div[1]/div[1]/div[1]/div[1]/div[2]').text
            except NoSuchElementException:
                location = None

            # Click on 'More' button to reveal additional applicant information
            more_button = WebDriverWait(driver, 2.5).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[3]/div/div[3]/div/main/div/div[1]/div[1]/div[1]/div[2]/div[3]'))
            )
            more_button.click()
            time.sleep(1)

            # Create a new BeautifulSoup object to parse updated page source
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('span', {'class': 'hiring-applicant-header-actions__more-content-dropdown-item-text'})
            time.sleep(1)
            # Extract LinkedIn URL
            try:
                linkedin = soup.find('div', class_='artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view p0').find('a').get('href')
                linked_url = 'https://www.linkedin.com/' + linkedin
            except AttributeError:
                linked_url = None
            time.sleep(1)
            # Extract resume link
            try:
                div_element = soup.find('div', class_='display-flex justify-space-between align-items-flex-start pl5 pr5 pt5 pb3')
                if div_element is None:
                    div_element = soup.find('div', class_='ui-attachment ui-attachment--doc')
                if div_element:
                    resume_link = div_element.find('a').get('href')
            except AttributeError:
                resume_link = None
            # Extract email and mobile number
            try: 
                email, mob = [r.text.replace('\n', '').replace(' ', '') for r in results]
            except ValueError:
                email = None
                mob =  None

            # Append applicant information to the list
            info = {'Name': name, 'Email': email, 'Mobile': mob,
                    'Location': location, "Current title": 
                    current_title,'Current Company': current_company,
                    'LinkedIn_URL': linked_url, 'Resume_Link': resume_link}
            
            #Masked email and phone
            masked_email = '*' * (len(email) - len(email)//2) + email[len(email)//2:]
            masked_phone = mob[:len(mob)//2] + '*' * (len(mob) - len(mob)//2)

            #Print Job applicant info
            print(f"Name: {name}\nEmail: {masked_email}\nMobile: {masked_phone}\nLocation: {location}\nCurrent title: {current_title}\nCurrent Company: {current_company}\nLinkedIn_URL: {linked_url}")

            applicants_info.append(info)


https://www.linkedin.com/hiring/jobs/3785030073/applicants/22377439383/detail/?r=UNRATED%2CGOOD_FIT%2CMAYBE&start=0
Name: Stewart Z Zote
Email: *********gmail.com
Mobile: +91708*******
Location: Imphal, Manipur, India
Current title: Virtual Assistant
Current Company: Full Spectrum
LinkedIn_URL: https://www.linkedin.com//in/stewart-z-zote-531158187/
Name: Ritu Guldagad
Email: ************rg@gmail.com
Mobile: +91876*******
Location: Pune, Maharashtra, India
Current title: Counsellor
Current Company: College Vidya
LinkedIn_URL: https://www.linkedin.com//in/ritu-guldagad-704b531b8/
Name: Surendar S
Email: ************rg@gmail.com
Mobile: +91876*******
Location: Pune, Maharashtra, India
Current title: Counsellor
Current Company: College Vidya
LinkedIn_URL: https://www.linkedin.com//in/ritu-guldagad-704b531b8/
Name: Sonal Suthar
Email: *************992@gmail.com
Mobile: +91820*******
Location: Jodhpur, Rajasthan, India
Current title: Executive Assistant
Current Company: Hst Hospitality
Linke

In [ ]:
# Create a DataFrame from the collected applicant information
applicants_df = pd.DataFrame(applicants_info)

# Save DataFrame to an Excel file
applicants_df.to_excel('job_applicants.xlsx')